In [9]:
import os
import sqlite3
import nltk
import random
import numpy as np 
from collections import defaultdict
from nltk import FreqDist
from nltk.corpus import stopwords
import nltk
from nltk import FreqDist
from string import punctuation
import string

In [6]:
big_file = open('/Users/steviodong/Desktop/TM/assignments/assignment-group-comparison-StevioD/big.txt').read()

In [10]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [11]:
punctuation = set(punctuation)
punctuation.add("’")

In [12]:
query_results = convention_cur.execute(
                            '''
                                SELECT text, party
                                FROM conventions
                                WHERE speaker != "Unknown"
                            ''')

In [13]:
convention_data = defaultdict(str)

for row in query_results :
    text, party = row

    text = "".join([ch for ch in text if ch not in punctuation])    
    text = [w.lower() for w in text.split() if w.isalpha()]
    
    convention_data[party] += " ".join(text) + " "

In [20]:
def get_patterns(text, num_words=10):
    
    text = text.split()
    text = [word for word in text if not isinstance(word, int)]
    text = [word.lower() for word in text]
    text = [word for word in text if word not in string.punctuation]
    
    total_tokens = len(text)
    
    unique_tokens = len(list(FreqDist(text).keys()))

    avg_token_len = sum([len(word) for word in text]) / len(text)
    
    lex_diversity = unique_tokens/total_tokens
    
    #top 10 words
    top_words = FreqDist(text).most_common(num_words)
        
    
    #Results 
    results = {'tokens':total_tokens,
               'unique_tokens':unique_tokens,
               'avg_token_length':avg_token_len,
               'lexical_diversity':lex_diversity,
               'top_words':top_words}
    
    return(results)

In [21]:
def group_comparison(corpus_1, corpus_2, num_words = 10, ratio_cutoff = 5):
    c1_results = get_patterns(corpus_1)
    c2_results = get_patterns(corpus_2)
    
    #Clean Corpus 1 
    corpus_1 = corpus_1.split()
    corpus_1 = [word for word in corpus_1 if not isinstance(word, int)]
    corpus_1 = [word.lower() for word in corpus_1]
    corpus_1 = [word for word in corpus_1 if word not in string.punctuation]


    #Corpus 2 
    corpus_2 = corpus_2.split()
    corpus_2 = [word for word in corpus_2 if not isinstance(word, int)]
    corpus_2 = [word.lower() for word in corpus_2]
    corpus_2 = [word for word in corpus_2 if word not in string.punctuation]

    c1_dict = {}
    for key, value in FreqDist(corpus_1).items():
        if value > ratio_cutoff:
            c1_dict[key] = value/len(corpus_1)
        else:
            continue

    c2_dict = {}
    for key, value in FreqDist(corpus_2).items():
        if value > ratio_cutoff:
            c2_dict[key] = value/len(corpus_2)
        else:
            continue

    c_1VSc_2 = {}
    c_2VSc_1 = {}
    
    #adding comparison to dictionaries
    for key in c1_dict.keys():
        if key in c2_dict.keys():
            c_1VSc_2[key] = c1_dict[key]/c2_dict[key]
            c_2VSc_1[key] = c2_dict[key]/c1_dict[key]
    
    #sorting dictionary
    #num words is the number of words to be displayed
    one_vs_two = sorted(c_1VSc_2.items(), key=lambda k: k[1], reverse = True)[:num_words]
    two_vs_one = sorted(c_2VSc_1.items(), key=lambda k: k[1], reverse = True)[:num_words]
    
    #Adding to dictionary
    comparison_dict = {'one': c1_results,
                      'two': c2_results,
                      'one_vs_two': one_vs_two,
                      'two_vs_one': two_vs_one}
    
    return(comparison_dict)

In [22]:
def group_comparison(corpus_1, corpus_2, num_words = 10, ratio_cutoff = 5):
    c1_results = get_patterns(corpus_1)
    c2_results = get_patterns(corpus_2)
    
    #Corpus 1 cleaning
    corpus_1 = corpus_1.split()
    corpus_1 = [word for word in corpus_1 if not isinstance(word, int)]
    corpus_1 = [word.lower() for word in corpus_1]
    corpus_1 = [word for word in corpus_1 if word not in string.punctuation]


    #Corpus 2 cleaning
    corpus_2 = corpus_2.split()
    corpus_2 = [word for word in corpus_2 if not isinstance(word, int)]
    corpus_2 = [word.lower() for word in corpus_2]
    corpus_2 = [word for word in corpus_2 if word not in string.punctuation]


    c1_dict = {}
    for key, value in FreqDist(corpus_1).items():
        if value > ratio_cutoff:
            c1_dict[key] = value/len(corpus_1)
        else:
            continue

    c2_dict = {}
    for key, value in FreqDist(corpus_2).items():
        if value > ratio_cutoff:
            c2_dict[key] = value/len(corpus_2)
        else:
            continue

    c_1VSc_2 = {}
    c_2VSc_1 = {}
    
    for key in c1_dict.keys():
        if key in c2_dict.keys():
            c_1VSc_2[key] = c1_dict[key]/c2_dict[key]
            c_2VSc_1[key] = c2_dict[key]/c1_dict[key]
    #sorting dic & display num word
    one_vs_two = sorted(c_1VSc_2.items(), key=lambda k: k[1], reverse = True)[:num_words]
    two_vs_one = sorted(c_2VSc_1.items(), key=lambda k: k[1], reverse = True)[:num_words]
    
    #compare dic

    comparison_dict = {'one': c1_results,
                      'two': c2_results,
                      'one_vs_two': one_vs_two,
                      'two_vs_one': two_vs_one}
    
    return(comparison_dict)

In [23]:
group_comparison(convention_data['Republican'], convention_data['Democratic'], num_words = 10, ratio_cutoff = 5)

{'one': {'tokens': 82930,
  'unique_tokens': 7151,
  'avg_token_length': 4.58812251296274,
  'lexical_diversity': 0.08622935005426263,
  'top_words': [('the', 3817),
   ('and', 2923),
   ('to', 2566),
   ('of', 1975),
   ('a', 1691),
   ('in', 1503),
   ('i', 1173),
   ('that', 1103),
   ('for', 1036),
   ('we', 1033)]},
 'two': {'tokens': 74142,
  'unique_tokens': 6122,
  'avg_token_length': 4.4159855412586655,
  'lexical_diversity': 0.08257128213428286,
  'top_words': [('the', 3067),
   ('and', 2729),
   ('to', 2490),
   ('of', 1771),
   ('a', 1608),
   ('we', 1190),
   ('in', 1153),
   ('our', 1075),
   ('that', 1066),
   ('for', 1043)]},
 'one_vs_two': [('freedom', 6.832952059396048),
  ('heroes', 6.168814662968769),
  ('trade', 5.811202218738695),
  ('greatest', 5.6196241236154405),
  ('bless', 5.364186663451104),
  ('cities', 5.364186663451103),
  ('police', 5.300327298410019),
  ('land', 5.201635552437433),
  ('grateful', 4.086999362629412),
  ('officers', 4.023139997588327)],
 

In [24]:
group_comparison(big_file, convention_data['Democratic'], num_words = 10, ratio_cutoff = 5)

{'one': {'tokens': 1095046,
  'unique_tokens': 75151,
  'avg_token_length': 4.861016797467869,
  'lexical_diversity': 0.06862816721854607,
  'top_words': [('the', 78172),
   ('of', 39450),
   ('and', 37009),
   ('to', 28295),
   ('in', 21393),
   ('a', 20610),
   ('that', 11483),
   ('he', 11481),
   ('was', 11159),
   ('his', 9961)]},
 'two': {'tokens': 74142,
  'unique_tokens': 6122,
  'avg_token_length': 4.4159855412586655,
  'lexical_diversity': 0.08257128213428286,
  'top_words': [('the', 3067),
   ('and', 2729),
   ('to', 2490),
   ('of', 1771),
   ('a', 1608),
   ('we', 1190),
   ('in', 1153),
   ('our', 1075),
   ('that', 1066),
   ('for', 1043)]},
 'one_vs_two': [('such', 13.367245889996532),
  ('which', 8.657059612107618),
  ('andrew', 6.993139492378794),
  ('looked', 5.642228728290866),
  ('began', 5.5820449551890965),
  ('may', 5.23880937421807),
  ('though', 4.566443784096741),
  ('soon', 4.525067440089275),
  ('between', 4.3264609888534356),
  ('eyes', 4.265524918587896)]